In [1]:
using CPUTime
using DelimitedFiles
using Base
using LightGraphs, SimpleWeightedGraphs
using Distributed
using Printf

In [2]:
addprocs(2)
nprocs()

3

In [3]:
function readDataFromFile(filename="in7.txt")
    open(filename, "r") do f 
        n, m = split(readline(f))
        n = parse(Int, n)
#         cnt=0
        sources = Vector{Int}()
        destinations = Vector{Int}()
        weights = Vector{Float64}()
        for ln in eachline(f)
#             a,b,c = split(ln)
            a,b,c = readdlm(IOBuffer(ln), Int)
            if(a==0)
                a=n
            end
            if(b==0)
                b=n
            end
#             cnt+=1
#             if(cnt%10000==0)
#                 println(cnt)
#             end
            if(c ==0)
                push!(sources,a)
                push!(destinations,b)
                push!(weights,0.000001)
            else
                push!(sources,a)
                push!(destinations,b)
                push!(weights,c)
            end
        end
        g = SimpleWeightedGraph(sources, destinations, weights)
        return g
    end
end
# for in_n_10e5_m_5e6.in, generate the file from graph_generator.ipynb
g = readDataFromFile("in_n_10e5_m_5e6.in")
# g = readDataFromFile("in7.txt")

{100000, 5000000} undirected simple Int64 graph with Float64 weights

In [4]:
# Baseline
@time @CPUtime (mst, weight) = boruvka_mst(g, minimize=false)
println(length(mst))
@printf("%.3lf", weight)

elapsed CPU time: 8.547239 seconds
  8.671573 seconds (664.64 k allocations: 1.551 GiB, 2.34% gc time)
99999
493973783578.000

In [9]:
include("Final/boruvka-singlethread.jl")

contractvertex (generic function with 3 methods)

In [10]:
@time @CPUtime (mst, weight) = boruvka_mst_singlethread(g)
println(length(mst))
@printf("%.3lf", weight)

Max iteration: 18
elapsed CPU time: 10.17709 seconds
 10.178529 seconds (573.27 k allocations: 57.629 MiB, 0.29% gc time)
99999
6018132840.000

In [7]:
include("Final/boruvka-multithread.jl")

contractvertex (generic function with 2 methods)

In [8]:
@time @CPUtime (mst, weight) = boruvka_mst_multithread(g, minimize=false)
println(length(mst))
@printf("%.3lf", weight)

Max iteration: 18
elapsed CPU time: 13.835071 seconds
  4.528539 seconds (457.98 k allocations: 52.344 MiB)
99999
493973783578.000

In [5]:
include("Final/boruvka-distributed.jl")

contractvertex (generic function with 1 method)

In [6]:
@time @CPUtime (mst, weight) = boruvka_mst_distributed(g, minimize=false)
println(length(mst))
@printf("%.3lf", weight)

Max iteration: 18
elapsed CPU time: 3.272268 seconds
 12.836661 seconds (7.48 M allocations: 402.431 MiB, 1.25% gc time)
99999
493973783578.000